## <center>Text translation</center>
In order to avoid several languages a translation process will be applied on the extracted text

In [1]:
import pandas as pd
import transformers
print('Transformer version',transformers.__version__)
df = pd.read_csv('../data/data_with_text_final.csv')
df.head()

Transformer version 4.11.3


,Unnamed: 0.1,Unnamed: 0,filename,url,description,type,filepath,scraped,old_filename,old_filepath,height,width,channels,landscape,text,lang_code,lang_conf,text_ocr
0,0,0,img_0000000.jpg,https://templates.invoicehome.com/modele-factu...,100 modèles gratuits de factures | Téléchargem...,facture,./final/img_0000000.jpg,1,img_0000000.jpg,./facture/img_0000000.jpg,1061.0,750.0,3.0,0.0,facture logo joanna binet coubertin paris fact...,fr,0.704567,FACTURE LOGO Joanna Binet 48 Coubertin 31400 ...
1,1,1,img_0000001.jpg,https://templates.invoicehome.com/modele-factu...,100 modèles gratuits de factures | Téléchargem...,facture,./final/img_0000001.jpg,1,img_0000001.jpg,./facture/img_0000001.jpg,1061.0,750.0,3.0,0.0,joanna binet coubertin paris facture facturé c...,fr,0.798047,Joanna Binet 48 Coubertin 31400 Paris FACTURE...
2,2,2,img_0000002.jpg,https://www.coover.fr/wp-content/uploads/2021/...,Facture proforma : modèle à télécharger gratui...,facture,./final/img_0000002.jpg,1,img_0000002.jpg,./facture/img_0000002.jpg,1024.0,726.0,3.0,0.0,facture entreprise société adresse postale adr...,fr,0.985557,FACTURE Mon entreprise : Nom de la société Ad...
3,3,3,img_0000003.jpg,https://templates.invoicehome.com/modele-factu...,100 modèles gratuits de factures | Téléchargem...,facture,./final/img_0000003.jpg,1,img_0000003.jpg,./facture/img_0000003.jpg,1061.0,750.0,3.0,0.0,joanna binet coubertin paris facture cendrillo...,fr,0.738264,Joanna Binet 48 Coubertin 31400 Paris FACTURE...
4,4,4,img_0000004.jpg,https://files.intum.com/v/vosfactures/files/31...,Base de Connaissances - Vos Factures - Créer u...,facture,./final/img_0000004.jpg,1,img_0000004.jpg,./facture/img_0000004.jpg,771.0,773.0,3.0,1.0,payer ligne facture sfideli date création date...,fr,0.575626,Payer en ligne > FACTURE No SFIDELI F/05.2020...


**Words count**

In [2]:
df['words_count'] = df['text_ocr'].apply(lambda x : len(str(x).split(' ')))
print('Max words count',df['words_count'].max())

Max words count 1565


**Normalize text before tanslate**

In [1]:
import string
import re

def replacement(match):
    return match.group(1).title()

def normalize_before_translate(text):
    text = re.sub(r'([A-ZÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]+)',replacement, text)
    text = re.sub("\d+[\.,/]?", "", text)
    text = re.sub("[<>*%]?", "", text)
    return text

In [2]:
normalize_before_translate('TEST <test> 12.5 a 12,5 12/12/222 RRRR 10% gfh')

'Test test  a   Rrrr  gfh'

**English to french translation**

In [3]:
from transformers import pipeline
en_fr_translator = pipeline('translation_en_to_fr')

def translate_en_to_fr(text,verbose=True):
    try:
        text = normalize_before_translate(text)
        result = en_fr_translator(text,max_length=2000)[0]['translation_text']
        if verbose:
            print('From : ',text[0:30],' ... To:',result[0:30],' ...')
        return result
    except Exception as e:
        print('Except on : ',text[0:70],str(e))
        return text

No model was supplied, defaulted to t5-base (https://huggingface.co/t5-base)


**French to english translation**

In [4]:
fr_en_translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")

def translate_fr_to_en(text,verbose=True):
    try:
        text = normalize_before_translate(text)
        result = fr_en_translator(text,max_length=2000)[0]['translation_text']
        if verbose:
            print('From : ',text[0:30],' ... To:',result[0:30],' ...')
      
    except:
        print('Except try by phrase ')
        phrases = text.split('.')
        result = ''
        for phrase in phrases:
            try:
                tphrase = fr_en_translator(phrase,max_length=2000)[0]['translation_text']
                if verbose:
                    print('From phrase: ',phrase[0:30],' ... To:',tphrase[0:30],' ...')
                result = result + tphrase + '.' 
            except:
                print('Except on phrase ',phrase[0:30])  

    return result
    

In [11]:
#Temporary code
import pandas as pd
import transformers
df = pd.read_csv('../data/data_with_text_translated.csv')
df[df['lang_code']=='en'].head()


,Unnamed: 0.1,Unnamed: 0,filename,url,description,type,filepath,scraped,old_filename,old_filepath,...,lang_code,lang_conf,text_ocr,words_count,text_en,text_fr,text_fr_norm,text_en_norm,target_en,target_fr
12,12,12,img_0000012.jpg,https://www.facture.net/blog/wp-content/upload...,Délai de facturation : quand faut-il envoyer l...,facture,./final/img_0000012.jpg,1,img_0000013.jpg,./facture/img_0000013.jpg,...,en,0.443806,INVOICE Brand Name : - - Invoice to: Dwyane C...,101,Invoice Brand Name : - - Invoice to: Dwyane C...,NaN,NaN,invoice brand name invoice dwyane clark dummy ...,invoice,facture
90,90,90,img_0000090.jpg,https://www.ville-pierrelaye.fr/sites/pierrela...,Carte Nationale d'Identité (CNI) | Site de la ...,id_pieces,./final/img_0000090.jpg,1,img_0000024.jpg,./id_pieces/img_0000024.jpg,...,en,0.091803,REPUBLIQUE FRANCAISE CARTE NATIONALE DIDENTHE...,30,Republique Francaise Carte Nationale Didenthe...,NaN,NaN,republique francaise carte nationale didenthet...,id piece,pièce identité
103,103,103,img_0000103.jpg,https://www.manche.gouv.fr/var/ezwebin_site/st...,Carte nationale d'identité / Démarches adminis...,id_pieces,./final/img_0000103.jpg,1,img_0000039.jpg,./id_pieces/img_0000039.jpg,...,en,0.239839,TAILLETHegN DATE: DEI DELIVRANCE/Det X issue:...,29,TaillethegN Date: Dei Delivrance/Det X issue:...,NaN,NaN,taillethegn date delivrance issue adressei add...,id piece,pièce identité
108,108,108,img_0000108.jpg,https://encrypted-tbn0.gstatic.com/images?q=tb...,Information importante sur la carte nationale ...,id_pieces,./final/img_0000108.jpg,1,img_0000045.jpg,./id_pieces/img_0000045.jpg,...,en,0.152425,- E 58765235012 - - KAKTIN nove FAXUIN FCHARL...,23,"- E - - Kaktin nove Faxuin Fcharles, Jean, P...",NaN,NaN,kaktin nove faxuin fcharles jean pierre necele...,id piece,pièce identité
112,112,112,img_0000112.jpg,https://cdn.paris.fr/paris/2022/02/16/huge-a53...,Carte nationale d'identité et passeport : comm...,id_pieces,./final/img_0000112.jpg,1,img_0000052.jpg,./id_pieces/img_0000052.jpg,...,en,0.070712,RÉPUBLQUEFRAN FRANCAISE IORNTTC CAAD RRMN MET...,20,Républquefran Francaise Iornttc Caad Rrmn Met...,NaN,NaN,républquefran francaise iornttc caad rrmn metg...,id piece,pièce identité


**Test on sample**

In [7]:
print('EGLISH TO FRENCH\n')
row = df[df['lang_code']=='en'].sample()
text = row['text_ocr'].values[0]
print('Source:',text)
print('\n\n')
print('Translated:',translate_en_to_fr(text,verbose=False))

print('\nFRENCH TO ENGLISH\n')
row = df[df['lang_code']=='fr'].sample()
text = row['text_ocr'].values[0]
print('Source:',text)
print('\n\n')
print('Translated:',translate_fr_to_en(text,verbose=False))


#print('Count fr :',df[df['lang_code']=='en'].shape[0])
#df.loc[df['lang_code']=='en',['text_fr']] = df[df['lang_code']=='en']['text'].apply(translate)


EGLISH TO FRENCH

Source:  DOMESTIC PRODUCT DEVELOPMENT IIL ESTABLISHED BRANDS KOOL BELAIR VFM CAPRI TAR/NIC REVIEW PROCESS IV. MACON SUPPORT CONSOLIDATION PRIMARYFAB SUPPORT DC/MACON COMPARISON 526030559



Translated: Développement des produits intérieurs Iil Marques établies Kool Belair Vfm Processus d'examen Capri Tar/Nic Iv. Macon Support Consolidation Primaryfab Support Dc/Macon Comparison

FRENCH TO ENGLISH

Source:  nom de rauto-entrepreneur FACTURE Adresse CP Ville Téléphone / Fax Références Internet Dispensé d'immatriculation au registre du commerce et des societes (RCS) et au répertoire des métiers Société et/ou Nom du client Adresse CP Ville Référence : Date N'client Intitulé: Description du projet et/ou Produit facturé Quantité 3 Désignation Prix unitaire HT Prix total HT Serviettes de plage 34 102 Total HT 102,00 € TVA non applicable, art. 293 B du CGI En votre aimable règlement, Cordialement, Conditions de paiement : paiement à réception de facture, à 30 jours... Aucun e

**Translate all dataframe en to fr**

In [12]:
df.loc[df['lang_code']=='en',['text_fr']] = df[df['lang_code']=='en']['text_ocr'].apply(translate_en_to_fr)

From :   Invoice Brand Name : - - Invo  ... To: Facture Marque Nom : - - Factu  ...
From :   Republique Francaise Carte Na  ... To: Republique française Carte Nat  ...
From :   TaillethegN Date: Dei Delivra  ... To: TaillethegN Date: Dei Delivran  ...
From :   - E  - - Kaktin nove Faxuin F  ... To: - E - - Kaktin nove Faxuin Fch  ...
From :   Républquefran Francaise Iornt  ... To: Républquefran Francaise Iorntt  ...
From :   - Ssed -  ... To: - Ssed -  ...
From :   Talleins m Adressei Athee sru  ... To: Talleins m Adressei Athee srue  ...
From :   Dcngns - - Cgonnor Caete Nati  ... To: Dcngns - - Nation Cgonnor Caet  ...
From :   Face Arriere De La Carte Alur  ... To: Face Arriere De La Carte Alur   ...
From :   Sae Cenndenoun - Edtnermret X  ... To: Sae Cenndenoun - Edtnermret XX  ...
From :   asr Tailleie Height  m Date D  ... To: asr Tailleie Height m Date De   ...
From :   Ilyal visage(s). matplotlib.i  ... To: Ilyal visage(s). matplotlib.im  ...
From :   Attestation Df Deplacement

Token indices sequence length is longer than the specified maximum sequence length for this model (517 > 512). Running this sequence through the model will result in indexing errors


From :   - Juu Prt  Garçadde Magalhaes  ... To: - Juu Prt Garçadde Magalhaes I  ...
From :   Union X. European Maxxaat Mam  ... To: Union X. European Maxxaat Mamm  ...
From :   Repubukahrvaiska Uiovnica Pas  ... To: Repubukahrvaiska Uiovnica Pass  ...
From :   - Li Lei International Passpo  ... To: - Li Lei Passeport internation  ...
From :   Visas Visas Download from S D  ... To: Visas Visas Téléchargement de   ...
From :   a  I a Diell RaRe alamy Image  ... To: a I a Diell RaRe alamy Image I  ...
From :   The Virgin Islands TypeType P  ... To: Îles Vierges TypeType P Suname  ...
From :   Unitedstatesofamerica Unitedt  ... To: Unitedstatesofamerica Unitedt   ...
From :   Geiting A Passport This Year?  ... To: Youf Psa brihontohcole : equro  ...
From :   Canada - - Can  Hartin Sarah   ... To: Canada - - Can Hartin Sarah -   ...
From :   Personal Data And Emergency C  ... To: Données personnelles et contac  ...
From :   Aze  ... To: Aze  ...
From :   gettyimages Bloomberg  -   ... To: g

From :   Ff Principal Investigator.Pro  ... To: Ff Principal Investigator.Prog  ...
From :   Copyright  by The Humana Pres  ... To: Copyright by The Humana Press   ...
From :   U. Eo.i JD ouv. Faiu. Lu risa  ... To: U. Eo.i JD ouv. Faiu. Lu risal  ...
From :   Cpn valie Blurd A  rl Atr pos  ... To: Cpn valie Blurd A rl Atr pos B  ...
From :   Vol.  No. Sounding Board  Ref  ... To: Vol. No. Sounding Board Refere  ...
From :   Basic Weight Specification Ef  ... To: Caractéristiques de base du po  ...
From :   Ju lane-l  :  Dr. R. B. Selig  ... To: R. B. Seligman H. B. Merritt e  ...
From :   S de cheche Universite sur ra  ... To: S de cheche Universite sur ram  ...
From :   Sept.   J.Immunology Identifi  ... To: J.Immunology Identification an  ...
From :   Dna Plant Echnology Ccrporati  ... To: Echnologie des plantes à dna R  ...
From :   Yardmark Survey () One Sponso  ... To: Sondage Yardmark () Un command  ...
From :   Inbifo Institut fur biologisc  ... To: Inbifo Institut fur biologis

From :   .  P Makersot Cigarettes Clai  ... To: . . . . . . . . . . . . . . .   ...
From :   Philip Morris Companiesinc, I  ... To: Philip Morris Companiesinc, In  ...
From :   Stockton Street Merit Ks Rebu  ... To: Stockton Street Merit Ks Rebul  ...
From :    Philip Morris "Takedown" Rep  ... To: Philip Morris "Takedown" Repor  ...
From :   __ppt Fgu/Uro The Electrophys  ... To: __ppt Fgu/Uro Les effets élect  ...
From :   Expenditure And Major Donor C  ... To: Dépenses et principaux donateu  ...
From :   Note for Little, Greg Molitor  ... To: Note for Little, Greg Molitor,  ...
From :   Survey Sheet WinsTon Lights &  ... To: Fiche de sondage WinsTon Light  ...
From :   Dorsey & Whitney Llp Po.Box    ... To: Dorsey & Whitney Llp Po.Box Mi  ...
From :   Ted Bates & Compar-v/New York  ... To: Ted Bates & Compar-v/New York,  ...
From :   Vol.  No.   Apri   Biochemica  ... To: Vol. No. Apri Biochemical And   ...
From :   Soft Pack 's Ailter, Menthol   ... To: Soft Pack 's Ailter, Menthol

...

From :   Fire Memorandun To: Mr. J. W.  ... To: M. J. W. Burgard Ce: M. E. P.   ...
From :   Pku Iaive vul I nu l  Carcino  ... To: Pku Iaive vul I nu l Carcinoge  ...
From :   Market Facts, Inc.I  : Ith St  ... To: Market Facts, Inc.I : Ith St.   ...
From :   Dallas Morning News Dallas, T  ... To: Dallas Morning News Dallas, Te  ...
From :   Biographical Sketch Name: Jam  ... To: Notice biographique Nom : Jame  ...


Your input_length: 3386 is bigger than 0.9 * max_length: 2000. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


From :   Leo Burnett u.S.A. Adivirion   ... To: Leo Burnett u.S.A. Adivirion O  ...
From :   Eating Right New! Improved! D  ... To:                                 ...
From :   Philip Morris Limited Inter-O  ... To: Memorandum Inter-Office de Phi  ...
From :   # S Ballot  Code  Name Ted Da  ... To: # S Code du bulletin Nom Ted D  ...
From :    : Rj Reynolds P.A. Letournea  ... To: : Rj Reynolds P.A. Letourneau   ...
From :   - Argus Research Laboratories  ... To: - Argus Research Laboratories,  ...
From :   Waco, Texas News-Tribune D.    ... To: Waco, Texas News-Tribune D. Wa  ...
From :   -idy -uer - -inr -idy -uer -   ... To: -idy -uer - -inr -idy -uer - T  ...
From :   Philip Morris Companies Inc,   ... To: Philip Morris Companies Inc, I  ...
From :   ) Manchester, Ct Journal Inqu  ... To: ) Manchester, Ct Journal Inqui  ...
From :   Title Smoke Research kinctic   ... To: Titre Smoke Research kinctic m  ...
From :   Progress Report s  P. Lcrilla  ... To: Rapports d'étape P. Lcrillar

From :   "Hobble" Ifyou Liked "Drag"!!  ... To: "Hobble" Ifyou Liked "Drag"!!   ...
From :    Piedmont fleet - - ar recirc  ... To: Piémont - - ar recirculation f  ...
From :   Newport Newport  Alive with p  ... To: Newport Newport Vive avec plai  ...
From :   Weekly Business Review Week E  ... To: Revue hebdomadaire de la semai  ...
From :    Flam / B Srsy Odessa,   Ngar  ... To: Flam / B Srsy Odessa, Ngar Siv  ...
From :   Rlck..latatyguta.com on  :: P  ... To: Rlck..latatyguta.com on :: Pm   ...
From :   Ted Bates  comp sy, inc./adve  ... To: Ted Bates comp sy, inc./annonc  ...
From :    Fri : Fax   Philip Morris R&  ... To: Fri : Fax Philip Morris R&D a   ...
From :   Spend a mildermoment the. wit  ... To: Vous pourrez passer un moment   ...
From :   Louisville Beh 'S Reg. S.P.-E  ... To: Louisville Beh 'S Reg. S.P.-E   ...
From :   Leu Burnett u.A. A Divisson o  ... To: Leu Burnett u.A. A Divisson oy  ...
From :   Parketing Information Systems  ... To: Parketing Information System

From :   A-Contra Costa Times Sunday,   ... To: A-Contra Costa Times Sunday, O  ...
From :   Mr. F. E. Resnik P.A. Elchorn  ... To: M. F. E. Resnik P.A. Elchorn T  ...
From :   (Y  Purex Industrial Material  ... To: (Y Purex Industrial Material S  ...
From :   The Ri. Ationali Bank Nashagt  ... To: The Ri. Ationali Bank Nashagto  ...
From :   Datum MorlC Va Rap Ironmental  ... To: Date MorlC Va Rap Ironmental -  ...
From :   Minois Retail Merchants Assoc  ... To: Minois Retail Merchants Associ  ...
From :   Female   :  Male -   S  - Sha  ... To: Femme : Homme - S - Shami RF I  ...
From :   (B&W) Protected By Minnesota   ... To: (B&W) Protected By Minnesota T  ...
From :   Ff Principal Investigator/Pro  ... To: Ff Chercheur principal/directe  ...
From :   thag   Hartford, Conn. Couran  ... To: Courant D. -S. Hartford Metrop  ...


**Translate all dataframe fr to en**

In [8]:
df.loc[df['lang_code']=='fr',['text_en']] = df[df['lang_code']=='fr']['text_ocr'].apply(translate_fr_to_en)

From :   Facture Logo Joanna Binet  Co  ... To: Invoice Logo Joanna Binet Coub  ...
From :   Joanna Binet  Coubertin  Pari  ... To: Joanna Binet Coubertin Paris I  ...
From :   Facture Mon entreprise : Nom   ... To: Invoice My company : Company n  ...
From :   Joanna Binet  Coubertin  Pari  ... To: Joanna Binet Coubertin Paris B  ...
From :   Payer en ligne  Facture No Sf  ... To: Pay online Invoice No Sfideli   ...
From :   Joanna Binet  Coubertin  Pari  ... To: Joanna Binet Coubertin Paris I  ...
From :   [Nom de l'entreprise] Facture  ... To: [Company Name] Invoice (Addres  ...
From :   Ivotre Societe] (Choisir une   ... To: Ivotre Societe] (Choose a dato  ...
From :   Identification du vendeur Nom  ... To: Vendor identification Name: :   ...
From :   Facture Joanna Binet  Coubert  ... To: Invoice Joanna Binet Coubertin  ...
From :   Facture - - Date de facaurato  ... To: Invoice - - Date de faauraton   ...
From :   B   Facture No  Nom de votre   ... To: B Invoice No Company name ad

Token indices sequence length is longer than the specified maximum sequence length for this model (565 > 512). Running this sequence through the model will result in indexing errors


From :   Facture Saucisses Sa N: - Cou  ... To: Invoice Sausisses Sa N: - Cour  ...
Except try by phrase 
From phrase:   N' de contrat N" de racture    ... To: No contract Racking number Add  ...
From phrase:   Ainsi, en vous authentifiant   ... To: Thus, by authenticating yourse  ...
From phrase:  I Nom Prenom Adresse Cpcommune  ... To: I Name First name Address Cpco  ...
From phrase:  sodif  ... To: sodif  ...
From phrase:  com (espoce obonnes) Veolio Ea  ... To: com (spoce obonnes) Veolia Eau  ...
From phrase:   Cout moyen de eau par utilisa  ... To: Average cost of water per use   ...
From :   Facture NoF Fa_  Mon Logo Ala  ... To: Invoice NoF Fa_ My Logo Attent  ...
From :   Pierre,   Saint remy Telephon  ... To: Pierre, Saint remy Telephone:   ...
From :   TechnoPme Sa E TechnoPme Sa R  ... To: TechnoPme Sa E TechnoPme Sa Ru  ...
Except try by phrase 
From phrase:   Réf  ... To: Ref  ...
From phrase:   Pdciabonné Client  Du Contrat  ... To: Pdci Subscriber Contract Clien  ...
Fro

From phrase:   Bruxelles Sokv  wh  Kh  kuh    ... To: Brussels Sokv wh Kh kuh Eurkiv  ...
From phrase:   consommation réactive  Eur In  ... To: Reactive consumption Eur Inclu  ...
From :   Facture  Imprimer Page précéd  ... To: Invoice Print Previous page Ne  ...
From :   Eau & Assainissement Non Coll  ... To: Water & Sanitation No Collecti  ...
From :    eDf Entreprises Vos contacts  ... To: eDf Companies Your contacts Yo  ...
From :    Republique Française Carte:   ... To: French Republic Carte: Nationa  ...
From :   L République Française Carte   ... To: L République Française Carte N  ...
From :   Francaise Republique  Nationa  ... To: French Republic French Nationa  ...
From :   République Française = - Cart  ... To: French Republic = - National I  ...
From :   Epublique Francais Carte Nati  ... To: Epublique Francais Carte Natio  ...
From :   Ministere De L'Interieur Libe  ... To: Ministere De l'Interieur Liber  ...
From :   Republique Française Carte Na  ... To: French Republic 

From :   Caise Nationalité Française    ... To: French nationality coffee shop  ...
From :   Fr - Fran Rp Nom: Prilipe. Hi  ... To: Fr - Fran Rp Name: Prilipe. Hi  ...
From :   - - Md Republique Desbebes No  ... To: - - MD Republique Desbebes Nam  ...
From :   - A Fra A N - Omblioutt A Lon  ... To: - A Fra A N - Omblioutt A Lonn  ...
From :   LeHtfi République Carte Natio  ... To: LeHtfi République Carte Nation  ...
From :   Repubi: cUrops a lique - opée  ... To: Repubi: cUrops a lique - openn  ...
From :   l République Cartei D'Identit  ... To: l République Cartei D'Identite  ...
From :   Carte D'Identite De Français.  ... To: Identite French card. Fact She  ...
From :   Francase Caate" Républiquefra  ... To: Francase Caate" Républiquefra   ...
From :   K Erubliqu E Francaise Carte   ... To: K Erubliqu E Francaise Nationa  ...
From :   / - - / - o  Que  DD - -ie /   ... To: / - - / - o Que DD - -ie / - -  ...
From :   P.L.)  Ar Mnir sr el bant Sig  ... To: P.L.) Ar Mnir sr el bant Sig

...

Except on phrase                   Part employeu
Except try by phrase 
From phrase:   Dkgarimnmntdehechaten Diracti  ... To: Dkgarimnmntdehechaten Diractio  ...
From phrase:   Rigime adatlonnal calan fanct  ... To: - a cotartank sompars sa eu te  ...
From phrase:   Les cotaction sat plateniss s  ... To: Cotaction sat flatniss s retam  ...
From phrase:   Lacontribufios: aociales Aren  ... To: Lacontribufios: aociales Arenc  ...
From phrase:   Cotaiona patromaler donmap po  ... To: Cotaiona patromaler donmap pou  ...
From phrase:   somagons hinalead der rmunest  ... To: somagons hinalad der rmunest m  ...
Except on phrase   Sappliment famttel det taheme
From phrase:   sces ea enctona egarts - sera  ... To: sces ea enctona egarts - sera   ...
From phrase:   Lan martents de hRre peut : n  ... To: Lan martents of hRre can: neeu  ...
From phrase:   Coltt sotal amplayuuricurue d  ... To: Coltt sotal amplayuuricurue du  ...
From phrase:    votra compoat bancair  ... To: will vote compoat banca

From :   Part Salariale Taux Part Patr  ... To: Share Salarial Rate Share Patr  ...
From :   Bulletin De Salare Paie du (I  ... To: Bulletin De Salare Paye of (In  ...
Except try by phrase 
From phrase:   Comprendre - E Tatl - - - ma   ... To: Understanding - E Tatl - - - m  ...
From phrase:   Certte, Pu, Peps  ... To: Certte, Pu, Peps  ...
From phrase:   Ca Agrege, Psyen () Votre gra  ... To: Ca Agrege, Psyen () Your rank   ...
From phrase:   Votre indice ()) Des griles a  ... To: Your index () Aromet grills co  ...
From phrase:   echelon I det  -as nome dess   ... To: echelon I det -as name des cer  ...
From phrase:   Pe Pa Purs Ch PsyEw: Retrouve  ... To: Pe Pa Purs Ch PsyEw: Find all   ...
From phrase:   de résidence (In) Selons votr  ... To: (In) Depending on your place o  ...
From phrase:   -mer et indemntes - Fae - - L  ... To: - - - - - - - - - - - - - - -   ...
From :   Decryptage:comprendre votre n  ... To: Decryption:comprehensive your   ...
From :   Lireet Mafiche Depaie  .

From :   e  :   -  ... To: e: -  ...
From :   Bulletin Depaie Matricule Ref  ... To: Bulletin Depaie Matricule Ref.  ...
From :   Horaire Taux Horaire Ial Mens  ... To: Schedule Rates Schedule Ial Me  ...
From :   Lire ma fiche I depaie Sociét  ... To: Read my sheet I pay Societe Ad  ...
Except try by phrase 
From phrase:   Bulletin de paie Imprimer For  ... To: Payroll Print Print Format Pre  ...
From phrase:   collective : Syntec Début con  ... To: collective : Syntec Start cont  ...
From phrase:    Montant Sal  ... To: Sal amount  ...
From phrase:   Taux Pat  ... To: Rate  ...
From phrase:    Montant Pat  ... To: Amount Pat  ...
From phrase:   Santé Sécurité sociale Complé  ... To: Health Social security Supplem  ...
From phrase:   sociale Réduction générale de  ... To: General reduction in contribut  ...
From phrase:   des cot  ... To: of the costs  ...
From phrase:   chômage et maladie Impôt sur   ... To: Unemployment and sickness Inco  ...
From phrase:  service-publi  ... To: Ser

From :   Carte Chère maman, Lelieu:/ci  ... To: Map Dear Mother, Lelieu:/here,  ...
From :    A - - dws sdeil. dob obonten  ... To: A - - dws sdeil. dob obontenpr  ...
From :   Omages du Midi. Couleurs et s  ... To: Omages du Midi. Colours and fl  ...
From :   Loeil - - : Photoghapmie Oloe  ... To: Eye - - : Photoghapmie Oloeil  ...
From :   Pari Trance Lopca : palair ou  ... To: Bet Trance Lopca : palair ounr  ...
From :   Foluitatinpo wotee Cusson mco  ... To: Foluitatinpo wotee Cusson mcou  ...
From :   Higdes-wat-me.s, Hoit  Chere   ... To: Higdes-wat-me.s, Hoit Chere Ma  ...
From :   Mon cher Harry, j'espère que   ... To: My dear Harry, I hope your hol  ...
From :   te : Tout peltte Cueprart Pce  ... To: T : All peltte Cueprart Pce :   ...
From :   Republique, Française ) Carte  ... To: Republic, French ) Postcard Ce  ...
From :   gille  ... To: girl  ...
From :   Anne Berest La carte postale   ... To: Anne Berest Postcard Pari Tran  ...
From :   Bretagne (Collection E. Hamon  ...

From :   Mmin A M eue Cemrasr dstn lou  ... To: Mmin A M eue Cemrasr dstn rent  ...
From :   Gn  :  Ccourant -  tictactrip  ... To: Gn : Current - tickactrip A -   ...
Except try by phrase 
From phrase:   Royaume-Unt Bourne Brighton m  ... To: United Kingdom Bourne Brighton  ...
From phrase:   de w) Saint-Malo St-Brieuc Br  ... To: de w) Saint-Malo St-Brieuc Bre  ...
From phrase:   Wrouen uise Reims Metzo Paris  ... To: Wrouen uise Reims Metzo Paris   ...
From phrase:   de la Loire La Rochellese Mul  ... To: La Rochellese Mulhouses Rhin S  ...
From phrase:   senevs Wilano are  Golfe  ... To: senevs Wilano are Gulf  ...
From phrase:   de Nouvelle- Aquitaine  odena  ... To: New Aquitaine odena Bologna en  ...
From :   D'Amour et deaw de Source  ... To: Of Love and Deaw of Source  ...
From :   Le  avril - Salut, ça val - M  ... To: April - Hi, it's good - Me, it  ...
From :   U Grrrraspraraaepaa eceeenaAn  ... To: U Grrrraspraaraaepaa eceeenaAn  ...
From :   Telechargezune Image Ouunlogo 

In [9]:
#df.to_csv('../data/data_temp.csv')
df.loc[df['lang_code']=='fr',['text_fr']] = df[df['lang_code']=='fr']['text_ocr'].apply(normalize_before_translate)
df.loc[df['lang_code']=='en',['text_en']] = df[df['lang_code']=='en']['text_ocr'].apply(normalize_before_translate)
df.head()

,Unnamed: 0.1,Unnamed: 0,filename,url,description,type,filepath,scraped,old_filename,old_filepath,...,width,channels,landscape,text,lang_code,lang_conf,text_ocr,words_count,text_en,text_fr
0,0,0,img_0000000.jpg,https://templates.invoicehome.com/modele-factu...,100 modèles gratuits de factures | Téléchargem...,facture,./final/img_0000000.jpg,1,img_0000000.jpg,./facture/img_0000000.jpg,...,750.0,3.0,0.0,facture logo joanna binet coubertin paris fact...,fr,0.704567,FACTURE LOGO Joanna Binet 48 Coubertin 31400 ...,98,Invoice Logo Joanna Binet Coubertin Paris Invo...,Facture Logo Joanna Binet Coubertin Paris F...
1,1,1,img_0000001.jpg,https://templates.invoicehome.com/modele-factu...,100 modèles gratuits de factures | Téléchargem...,facture,./final/img_0000001.jpg,1,img_0000001.jpg,./facture/img_0000001.jpg,...,750.0,3.0,0.0,joanna binet coubertin paris facture facturé c...,fr,0.798047,Joanna Binet 48 Coubertin 31400 Paris FACTURE...,97,Joanna Binet Coubertin Paris Invoice Invoice i...,Joanna Binet Coubertin Paris Facture Factur...
2,2,2,img_0000002.jpg,https://www.coover.fr/wp-content/uploads/2021/...,Facture proforma : modèle à télécharger gratui...,facture,./final/img_0000002.jpg,1,img_0000002.jpg,./facture/img_0000002.jpg,...,726.0,3.0,0.0,facture entreprise société adresse postale adr...,fr,0.985557,FACTURE Mon entreprise : Nom de la société Ad...,45,Invoice My company : Company name Mail address...,Facture Mon entreprise : Nom de la société Ad...
3,3,3,img_0000003.jpg,https://templates.invoicehome.com/modele-factu...,100 modèles gratuits de factures | Téléchargem...,facture,./final/img_0000003.jpg,1,img_0000003.jpg,./facture/img_0000003.jpg,...,750.0,3.0,0.0,joanna binet coubertin paris facture cendrillo...,fr,0.738264,Joanna Binet 48 Coubertin 31400 Paris FACTURE...,99,Joanna Binet Coubertin Paris Bill To Cinderell...,Joanna Binet Coubertin Paris Facture À Cend...
4,4,4,img_0000004.jpg,https://files.intum.com/v/vosfactures/files/31...,Base de Connaissances - Vos Factures - Créer u...,facture,./final/img_0000004.jpg,1,img_0000004.jpg,./facture/img_0000004.jpg,...,773.0,3.0,1.0,payer ligne facture sfideli date création date...,fr,0.575626,Payer en ligne > FACTURE No SFIDELI F/05.2020...,157,Pay online Invoice No Sfideli F/ Creation date...,Payer en ligne Facture No Sfideli F/ Date de...


**Delete stop words**

In [10]:
import numpy as np
from nltk.corpus import stopwords
stop_words = {'en' : set(stopwords.words('english')),
              'fr':set(stopwords.words('french')),
              'it':set(stopwords.words('italian')),
              'de':set(stopwords.words('german')),
              'nl':set(stopwords.words('dutch')),
              'es':set(stopwords.words('spanish')),
              'pt':set(stopwords.words('portuguese'))
             }


from nltk.tokenize.regexp import RegexpTokenizer
tokenizer = RegexpTokenizer("[a-zA-ZàâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]{4,}")

from nltk.tokenize import word_tokenize


def normalize_text(txt,lang_code):
    if(lang_code not in ['fr', 'it', 'de', 'nl', 'es', 'pt']):
        lang_code='en'
    result = []   
    txt = str(txt).lower()
    words = tokenizer.tokenize(txt)
    if len(words) > 0:
        for word in words:
            if word not in stop_words[lang_code]:
                result.append(word)
        return " ".join(result)
    else:
        return np.nan

df['text_fr_norm'] = df['text_fr'].apply(lambda x:normalize_text(x,'fr'))
df['text_en_norm'] = df['text_en'].apply(lambda x:normalize_text(x,'en'))

In [11]:
df[['lang_code','text_fr_norm','text_en_norm']].head()

,lang_code,text_fr_norm,text_en_norm
0,fr,facture logo joanna binet coubertin paris fact...,invoice logo joanna binet coubertin paris invo...
1,fr,joanna binet coubertin paris facture facturé c...,joanna binet coubertin paris invoice invoice c...
2,fr,facture entreprise société adresse postale adr...,invoice company company name mail address tele...
3,fr,joanna binet coubertin paris facture cendrillo...,joanna binet coubertin paris bill cinderella a...
4,fr,payer ligne facture sfideli date création date...,online invoice sfideli creation date validity ...


**Translate labels in english and french**

In [12]:
targets = {
 'facture':{'fr':'facture','en':'invoice'},
 'id_pieces':{'fr':'pièce identité','en':'id piece'},
 'justif_domicile':{'fr':'justif domicile','en':'residence proof'},
 'passeport':{'fr':'passeport','en':'passport'},
 'paye':{'fr':'paye','en':'pay'},
 'carte postale':{'fr':'carte postale','en':'postcard'},
 'form':{'fr':'formulaire','en':'form'},
 'scientific_publication':{'fr':'publication scientifique','en':'scientific publication'},
 'advertisement':{'fr':'publicité','en':'advertisement'},
 'letter':{'fr':'lettre','en':'letter'},
 'resume':{'fr':'resume','en':'resume'},
 'specification':{'fr':'spécification','en':'specification'},
 'handwritten':{'fr':'ecrit main','en':'handwritten'},
 'memo':{'fr':'note','en':'memo'},
 'invoice':{'fr':'facture','en':'invoice'},
 'budget':{'fr':'budget','en':'budget'},
 'news_article':{'fr':'article actualité','en':'news article'},
 'presentation':{'fr':'presentation','en':'presentation'},
 'file_folder':{'fr':'dossier fichier','en':'file folder'},
 'scientific_report':{'fr':'rapport scientifique','en':'scientific report'},
 'email':{'fr':'email','en':'email'},
 'questionnaire':{'fr':'questionnaire','en':'questionnaire'}
}

df['target_en'] = df['type'].apply(lambda x:targets[x]['en'])
df['target_fr'] = df['type'].apply(lambda x:targets[x]['fr'])
df[['type','target_en','target_fr']].head(20)

,type,target_en,target_fr
0,facture,invoice,facture
1,facture,invoice,facture
2,facture,invoice,facture
3,facture,invoice,facture
4,facture,invoice,facture
5,facture,invoice,facture
6,facture,invoice,facture
7,facture,invoice,facture
8,facture,invoice,facture
9,facture,invoice,facture


**Save dataframe forthe next step**

In [13]:
df.to_csv('../data/data_with_text_translated.csv',index=False)